## Installing OpenPose Repository

In [1]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt

## Install system dependencies to run OpenPose

In [2]:
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

## Mount Google Drive

In [1]:
#mount google drive
from google.colab import drive
import glob
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Feature Extraction

## Folder Directories

In [4]:
burglar_directory = "/content/gdrive/My Drive/ICU/Dataset/Burglar"
normal_directory = "/content/gdrive/My Drive/ICU/Dataset/Normal"

burglar_data = '/content/gdrive/My\ Drive/ICU/Dataset/Burglar/'
normal_data = '/content/gdrive/My\ Drive/ICU/Dataset/Normal/'

burglar_output_directory = '/content/gdrive/My\ Drive/ICU/Output/Burglar_Json_v1/'
normal_output_directory = '/content/gdrive/My\ Drive/ICU/Output/Normal_Json_v1/'

## Extract burglar dataset skeleton Json

In [5]:
for video in os.listdir(burglar_directory):
   InputVideo = burglar_data + video
   folder = burglar_output_directory + video[0:3]
   !mkdir {folder}
   !cd openpose && ./build/examples/openpose/openpose.bin --video {InputVideo}  --write_json  {folder}  --display 0 --render_pose 0

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 31.488840 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 25.519220 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 29.430675 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 13.798192 seconds.
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto

## Extract normal dataset skeleton Json

In [6]:
for video in os.listdir(normal_directory):
   InputVideo = normal_data + video
   folder = normal_output_directory + video[1:4]
   print(folder)
   !cd openpose && ./build/examples/openpose/openpose.bin --video {InputVideo}  --write_json {folder} --display 0 --render_pose 0

/content/gdrive/My\ Drive/ICU/Output/Normal_Json_v1/006
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 24.375195 seconds.
/content/gdrive/My\ Drive/ICU/Output/Normal_Json_v1/035
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 16.831683 seconds.
/content/gdrive/My\ Drive/ICU/Output/Normal_Json_v1/036
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 12.228662 seconds.
/content/gdrive/My\ Drive/ICU/Output/Normal_Json_v1/037
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detect

# Feature Preparation


In [2]:
import numpy as np
import os
import cv2
import csv
import json
import pandas as pd


In [3]:
# instanciate dataframes 
body_keypoints_df = pd.DataFrame()

temp = []
labels = []
names = []

In [8]:
burglar_json = '/content/gdrive/My Drive/ICU/Output/Burglar_Json_v1/'
normal_json = '/content/gdrive/My Drive/ICU/Output/Normal_Json_v1/'

###Function to extract skeletal joint from Json

In [14]:
def extractSkeletonJoint(data,fileName,label):
  #Blank frames are skipped
  if np.array(data ['people']).__len__() != 0:
        keypoints = np.array(data ['people'] [0] ['pose_keypoints_2d']);
        temp.append(keypoints)
        #Pandas skips empty frames hence the label is adjusted
        names.append(fileName)
        labels.append(label)

## Function Create Data Frames

In [13]:
def createDataFrames(data, fileName, label):
  dataframe = pd.DataFrame(data)
  #append name and label of each frame
  dataframe = dataframe.assign(video=fileName,label=label)
  #assign 0 to NaN
  dataframe.fillna(0)
  return dataframe

## Burglar Json files to dataframes

In [9]:
try:
  for folder_name in sorted(os.listdir(burglar_json)):
      folder = sorted(os.listdir(burglar_json + folder_name))
      for file in folder:
         data = json.load(open(burglar_json+folder_name+ '/'+file))
         extractSkeletonJoint(data,file,'Burglar')
except BaseException as e:
  print("An exception  occurred at " + file + str(e))

In [17]:
print(len(temp))
print(len(labels))
print(len(names))

29023
29023
29023


## Write dataframes to csv

In [15]:
body_keypoints_df = createDataFrames(temp,names,labels)
burglar_csv_output = '/content/gdrive/MyDrive/ICU/Output/All/V2-original/burglar.csv'
body_keypoints_df.to_csv (burglar_csv_output, index = False, header=False)

## Normal Json files to dataframes


In [16]:
body_keypoints_df = pd.DataFrame()
temp = []
labels = []
names = []


try:
  for folder_name in sorted(os.listdir(normal_json)):
      folder = sorted(os.listdir(normal_json + folder_name))
      for file in folder:
         print(folder_name)
         data = json.load(open(normal_json+folder_name+ '/'+file))
         extractSkeletonJoint(data,file,'Normal')
except BaseException as e:
  print("An exception  occurred at " + file + str(e))


Streaming output truncated to the last 5000 lines.
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
180
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
181
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
182
1

## Write dataframes to csv

In [18]:
body_keypoints_df = createDataFrames(temp,names,labels)
normal_csv_output = '/content/gdrive/MyDrive/ICU/Output/All/V2-original/normal.csv'
body_keypoints_df.to_csv (normal_csv_output, index = False, header=False)